In [ ]:
!pip install pypdf autogen

In [2]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin


--2025-03-27 12:53:36--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.109.153, 185.199.111.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0.001s  

2025-03-27 12:53:36 (4.91 MB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/1f33a7ab/releases/pybi

ModuleNotFoundError: No module named 'dolfin'

In [3]:
# from google.colab import userdata
# GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [4]:
import pypdf
import autogen

In [5]:
# Setting Configuration for the llm
config_list = [{
        "model": "gemini-1.5-pro",
        "api_key": GOOGLE_API_KEY,
        "api_type": "google"
    }]

In [6]:
# Constructing the agents
gemini_config = [{
    "cache_seed" : 90,
    "temperature" : 1.0,
    "config_list" : config_list,
    "timeout" : 120,
}]

In [7]:
# Creating an assistant agent that codes the solutions and revises the code from the output given by userproxy.
assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=gemini_config[0],
)

# Creating a user-proxy that runs the code and provides feedback
userproxy = autogen.UserProxyAgent(
    name="User_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=100000,
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    code_execution_config={
        "work_dir": "Mechanics",
        "use_docker": False,
    }
)

In [10]:
initial_task = (
    "A 1m-by-1m elastic plate has Young's modulus of 1GPa and Poisson ratio of 0.3. It has zero displacement on the left edge and 0.1m displacement along the x direction on the right edge. Solve for the displacement using matplotlib with a proper plan of course of actions to taken by you and store the 3D displacement result in a PNG file."
)

In [11]:
chat_response = userproxy.initiate_chat(assistant, message=initial_task)


User_proxy (to assistant):

A 1m-by-1m elastic plate has Young's modulus of 1GPa and Poisson ratio of 0.3. It has zero displacement on the left edge and 0.1m displacement along the x direction on the right edge. Solve for the displacement using matplotlib with a proper plan of course of actions to taken by you and store the 3D displacement result in a PNG file.

--------------------------------------------------------------------------------
assistant (to User_proxy):

Here's a plan to solve for the displacement of the elastic plate using Python and matplotlib:

**Step 1: Install necessary libraries.** We'll need `numpy` for numerical calculations, `matplotlib` for plotting, and `scipy` for solving the partial differential equation. I'll provide a script to install these.

```python
# filename: install_dependencies.py
import subprocess
import sys

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

install("numpy")
install("matplotlib")
i

In [ ]:
from IPython.display import Image
Image("Mechanics/displacement.png")  # Ensure the path is correct


In [9]:
followup_tasks = ["Let's change the boundary condition on the right edge to a shear case, i.e., the displacement along y direction is 0.1m on the right edge. Please refine the mesh to 50-by-50 elements, solve the problem again and save result into another png file.",
                  "Let's also calculate the stress component sigma_xy and save it into another png file.",
                  "You plotted von Mises stress, not stress component sigma_xy. Please correct.",
                  "Let's fill in the hole with another material of Young's mudulus 10GPa. Please solve the problem and save displacement and stress sigma_xy in png files."]

In [10]:
for task in followup_tasks:
    chat_response = userproxy.send(message=task, recipient=assistant)
    print(" -------------------------------------------------------------------- TASK COMPLETED ---------------------------------------------------------------------")

User_proxy (to assistant):

Let's change the boundary condition on the right edge to a shear case, i.e., the displacement along y direction is 0.1m on the right edge. Please refine the mesh to 50-by-50 elements, solve the problem again and save result into another png file.

--------------------------------------------------------------------------------
assistant (to User_proxy):

Here's the updated plan:

1. **Modify boundary conditions:** Change the right edge boundary condition to reflect the shear displacement in the y-direction.

2. **Refine the mesh:** Create a 50x50 grid for the x and y coordinates.

3. **Solve the equations:** We will now need to solve a 2D elasticity problem.  Because an analytical solution is more challenging for this scenario, I will utilize an approximate solution by directly integrating the stress equilibrium equation since the stress/strain relationship is linear.

4. **Visualize the result:** Plot the displacement in the y-direction as a 3D surface plot